In [2]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
!pip install folium
import folium

     |████████████████████████████████| 92kB 8.7MB/s eta 0:00:011


In [3]:
# Fousquare credentials
CLIENT_ID = 'Q2XEESRF5D10GO1QZDRIPRKUYWBEZ3KODM32LY052VZFZ2VQ' 
CLIENT_SECRET = 'RFNK5OCOK1SRLNRV4JQ1LUQEFFONTN21UGHTSEXN4WVGSLEZ' 
VERSION = '20200501'

In [4]:
LIMIT = 500 # Maximum is 100
city = "New York, NY"
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, city, LIMIT,"4bf58dd8d48988d1e0931735") 
# category id contains the id for coffee shops
results = requests.get(url).json()

In [5]:
venues = json_normalize(results['response']['groups'][0]['items'])
df_venues = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
df_venues.columns = ['Name', 'Address', 'Lat', 'Lng']

Foursquare only gives us the nearest 100 venues in the city.

### Now let's map these coffee shops 

In [6]:
city_lat = np.mean([results['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results['response']['geocode']['geometry']['bounds']['sw']['lat']])
city_lng = np.mean([results['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results['response']['geocode']['geometry']['bounds']['sw']['lng']])
map = folium.Map(location=[city_lat, city_lng], zoom_start=11)

for lat, lng, label in zip(df_venues['Lat'], df_venues['Lng'], df_venues['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)  
print(f"Total number of coffee shops in {city} = ", results['response']['totalResults'])
print("Showing Top 100")

Total number of coffee shops in New York, NY =  252
Showing Top 100


In [7]:
map

In the map, we can see the city is dense with coffee shops. Now let's have a concrete measure of this density by getting the mean coordinate of all these coffee shop markers, and taking the average of the distance of venues to the mean coordinate.

In [9]:
city_lat = np.mean([results['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results['response']['geocode']['geometry']['bounds']['sw']['lat']])
city_lng = np.mean([results['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results['response']['geocode']['geometry']['bounds']['sw']['lng']])
map = folium.Map(location=[city_lat, city_lng], zoom_start=11)
venues_mean_coor = [df_venues['Lat'].mean(), df_venues['Lng'].mean()] 

for lat, lng, label in zip(df_venues['Lat'], df_venues['Lng'], df_venues['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(map)
    
label = folium.Popup("Mean Co-ordinate", parse_html=True)
folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)


print("Mean Distance from Mean coordinates")
print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[['Lat','Lng']].values)))

Mean Distance from Mean coordinates
0.025294035751580533


In [10]:
map

The mean location can be taken as a point which will be near to most of the venues. Thus it would be best for the tourists to book their hotel near that mean coordinate to surround themselves with most of these coffee shops.